In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
# wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
#                            rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

Please create a subscriber to the marker


## Door

In [4]:
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [5]:
door_s, hinge_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door_s.geometry.dims[0]/2-0.01-0.001,-door_s.geometry.dims[1]/2+0.05,0), 
                          rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

lever_s, knob_s = add_lever(pscene, knob, door_s, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [6]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [7]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=30)
tplan.prepare()

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [8]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True
def explicit_rule(pscene, node, leaf):
    if node[-2:] == ("r2", "knob"):
            return False
    if node[-2] != leaf[-2] and node[-1] != "grip0":
            return False
    if node[-1]=="grip0" and leaf[-1]=="knob" and node[-2]!="r1":
            return False
    if node[1] != leaf[1] and node[0] != "lever":
            return False
    if node[0] == "lever" and node[-2] != leaf[-2]:
            return False
    if node[:2] != leaf[:2] and node[-2]!="r2":
            return False
    return True
tplan.explicit_rule = explicit_rule

### Plan

In [9]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['door', 'door_frame', 'knob', 'lever']
('door_frame', 'h0', 'r1', 'knob')


In [10]:
goal_nodes = [('door_frame', 'h1', 'r1', 'knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=True)

Use 18/36 agents
list index out of range
try: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
error: 1.8
try transition motion
try: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
error: 2.0
try transition motion
transition motion tried: True
result: 0 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0') = success
branching: 0->1 (0.2/100.0 s, steps/err: 57(120.800018311 ms)/0.00144959310989)
try: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
result: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
branching: 1->2 (0.23/100.0 s, steps/err: 1(7.2329044342 ms)/0)
try: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
error: 0.73
joint max
constrained motion tried: False
Motion Plan Failure
try: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
transition motion tried: True
error: 0.73
result: 

constrained motion tried: False
joint max
try: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
error: 1.5
result: 2 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0') = success
transition motion tried: True
result: 1 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = fail
try: 9 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
try: 17 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
constrained motion tried: False
try transition motion
try transition motion
error: 0.73
try: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
list index out of range
Motion Plan Failure
Motion Plan Failure
branching: 2->17 (0.69/100.0 s, steps/err: 81(278.13410759 ms)/0.00167663572903)
list index out of range
result: 17 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
result: 7 - ('door_frame', 'h0', 'r1', 'knob')->('do

error: 0.73
try: 27 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
result: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
error: 0.73
error: 0.025
error: 0.73
error: 0.73
branching: 3->34 (0.89/100.0 s, steps/err: 1(14.7500038147 ms)/0)
try: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
try transition motion
transition motion tried: True
branching: 20->32 (0.89/100.0 s, steps/err: 1(30.0381183624 ms)/0)
try: 33 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
joint max
result: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = fail
list index out of range
error: 0.73
result: 5 - ('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0') = success
transition motion tried: True
constrained motion tried: False
try: 33 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
result: 3 - ('door_frame', 'h0', 'r1', 'g

list index out of range
transition motion tried: True
end
result: 3 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
constrained motion tried: True
branching: 3->50 (1.08/100.0 s, steps/err: 1(6.79111480713 ms)/0)
result: 49 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
branching: 49->51 (1.09/100.0 s, steps/err: 1(4.47797775269 ms)/0)
try: 39 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
end
Motion Plan Failure
result: 27 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = fail
end
result: 9 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
error: 0.73
branching: 9->52 (1.1/100.0 s, steps/err: 53(384.105920792 ms)/8.85081603275e-06)
try: 52 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
constrained motion tried: True
constrained motion tried: True
try: 53 - ('door_frame', 'h0', 'r2', 'grip0')->('door_fr

error: 0.73
try: 39 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
try: 41 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
result: 33 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
try: 9 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
branching: 33->67 (1.29/100.0 s, steps/err: 53(378.374099731 ms)/8.95554373219e-06)
error: 1.1
result: 39 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
list index out of range
joint max
result: 53 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
result: 9 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob') = success
try: 67 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
branching: 39->69 (1.31/100.0 s, steps/err: 1(7.08508491516 ms)/0)
branching: 53->68 (1.31/100.0 s, steps/err: 1(7.43293762207 ms)/0)
branching: 9->70 (1.31/100.0 s, steps/e

...

end
branching: 56->139 (2.52/100.0 s, steps/err: 1(6.19983673096 ms)/0)
Motion Plan Failure
error: 2.1
result: 79 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0') = fail
result: 120 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
error: 2.1
constrained motion tried: False
constrained motion tried: True
try: 107 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
Motion Plan Failure
branching: 120->140 (2.53/100.0 s, steps/err: 1(4.85491752625 ms)/0)
result: 62 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
result: 96 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0') = fail
try: 141 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
try: 41 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
transition motion tried: True
result: 107 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = success
joint max
branching: 62

Motion Filer Failure: ReachChecker
Motion Plan Failure
result: 129 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0') = fail
result: 43 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
try: 96 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
try: 147 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
error: 1.1
try: 69 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
result: 72 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
Motion Filer Failure: ReachChecker
error: 2.7
try: 46 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
result: 46 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0') = fail
Motion Filer Failure: ReachChecker
result: 147 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0') = fail
try: 53 - ('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
try: 43 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r

try: 160 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
error: 0.73
try: 157 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
error: 0.73
error: 0.73
try: 158 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
try: 158 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
try: 158 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
try: 159 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
error: 0.73
error: 0.73
joint max
error: 0.73
try: 157 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
error: 0.73
try: 161 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
constrained motion tried: False
error: 0.73
error: 0.73
result: 157 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = success
result: 157 - ('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = success
bran

try: 177 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 160->179 (3.41/100.0 s, steps/err: 53(332.971096039 ms)/1.17909706935e-05)
end
++ adding return motion to acquired answer ++
++ adding return motion to acquired answer ++
end
constrained motion tried: True
branching: 162->180 (3.41/100.0 s, steps/err: 53(239.083051682 ms)/1.17909706935e-05)
Goal reached
try: 179 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 177 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
constrained motion tried: True
try: 178 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
end
end
end
branching: 177->181 (3.43/100.0 s, steps/err: 1(7.56502151489 ms)/0)
result: 160 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
constrained motion tried: True
constrained motion tried: True
constrained motion tried: True
result: 160 - ('door_frame', 'h1', 'r

Goal reached
constrained motion tried: True
try: 200 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 164 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
Goal reached
branching: 164->203 (3.58/100.0 s, steps/err: 53(310.572147369 ms)/1.17909706935e-05)
try: 203 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
Goal reached
++ adding return motion to acquired answer ++
result: 200 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
Goal reached
result: 203 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
branching: 200->204 (3.6/100.0 s, steps/err: 1(7.48491287231 ms)/0)
branching: 203->205 (3.6/100.0 s, steps/err: 1(7.26985931396 ms)/0)
++ adding return motion to acquired answer ++
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
try: 179 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')


In [11]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [12]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'grip0')
('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
('door_frame', 'h0', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
('lever', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'knob')


### push back

In [13]:
term_state = snode_schedule[-1].state
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['door', 'door_frame', 'knob', 'lever']
('door_frame', 'h1', 'r1', 'knob')


In [17]:
goal_nodes = [('door_frame', 'h0', 'r1', 'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
list index out of range
try: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
error: 2.2
try transition motion
try: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
error: 2.5
try transition motion
try: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
error: 2.2
transition motion tried: True
try transition motion
result: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
branching: 0->1 (0.23/30.0 s, steps/err: 63(142.806053162 ms)/0.00168104255382)
transition motion tried: True
try: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 0 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
branching: 0->2 (0.27/30.0 s, steps/err: 83(139.667987823 ms)/0.00186949996488)
try: 2 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 1 - ('door_frame', 'h1', 'r1', 'grip0')

try transition motion
error: 0.73
error: 0.73
branching: 1->14 (0.66/30.0 s, steps/err: 1(17.1160697937 ms)/0)
result: 11 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
result: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
list index out of range
error: 0.73
result: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
try: 17 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 1->15 (0.68/30.0 s, steps/err: 1(28.529882431 ms)/0)
list index out of range
branching: 11->17 (0.69/30.0 s, steps/err: 25(133.297204971 ms)/0.00197128431114)
branching: 1->16 (0.7/30.0 s, steps/err: 1(22.5219726562 ms)/0)
try: 2 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
transition motion tried: True
try: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 17 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame

joint min
result: 4 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
try: 30 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 4->30 (0.97/30.0 s, steps/err: 28(358.701944351 ms)/0.00154055526651)
result: 29 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
collision
branching: 21->28 (0.97/30.0 s, steps/err: 1(7.38906860352 ms)/0)
try: 1 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
constrained motion tried: False
branching: 29->31 (0.97/30.0 s, steps/err: 1(8.04209709167 ms)/0)
Motion Plan Failure
try transition motion
result: 30 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
try: 2 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
joint min
branching: 30->32 (0.99/30.0 s, steps/err: 1(8.95094871521 ms)/0)
constrained motion tried: False
list index out of range
constrained motion tried: False


try: 43 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
branching: 36->41 (1.15/30.0 s, steps/err: 1(7.72905349731 ms)/0)
branching: 39->46 (1.16/30.0 s, steps/err: 1(13.9830112457 ms)/0)
try: 44 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
try: 45 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
end
result: 2 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = fail
try: 21 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
error: 0.73
list index out of range
try: 44 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
error: 0.73
error: 0.73
result: 38 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
try: 43 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
try: 45 - ('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
constrained motion tried: True
branching: 38->47 (1

try: 53 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
error: 0.097
try transition motion
end
constrained motion tried: True
end
constrained motion tried: True
result: 17 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0') = success
branching: 17->62 (1.41/30.0 s, steps/err: 53(393.962860107 ms)/3.70430459337e-05)
transition motion tried: True
joint min
result: 44 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
constrained motion tried: False
try: 62 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
branching: 44->63 (1.42/30.0 s, steps/err: 53(254.328966141 ms)/2.59094317883e-05)
result: 53 - ('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0') = success
error: 0.73
Motion Plan Failure
branching: 53->64 (1.43/30.0 s, steps/err: 4(54.0840625763 ms)/0.00137435748623)
try: 64 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
result: 52

try: 78 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
constrained motion tried: False
result: 60 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
collision
branching: 60->79 (1.69/30.0 s, steps/err: 53(324.645042419 ms)/8.26705641884e-06)
result: 78 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
constrained motion tried: False
result: 77 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
Motion Plan Failure
branching: 78->81 (1.69/30.0 s, steps/err: 1(9.79590415955 ms)/0)
try: 55 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
branching: 77->80 (1.69/30.0 s, steps/err: 1(7.54618644714 ms)/0)
Motion Plan Failure
result: 51 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
result: 51 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = fail
end
try: 80 - ('door_frame', 'h0', 'r2', 'grip0')->('lever

try: 94 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
try: 51 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h3', 'r2', 'grip0')
Motion Plan Failure
Motion Plan Failure
Motion Plan Failure
result: 93 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob') = success
error: 0.73
Motion Filer Failure: ReachChecker
error: 0.73
result: 51 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0') = fail
branching: 93->96 (1.89/30.0 s, steps/err: 1(25.0239372253 ms)/0)
result: 95 - ('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
error: 0.73
result: 73 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
try: 45 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0')
result: 52 - ('lever', 'h1', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
try: 90 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
try: 51 - ('lever', 'h1', 'r2', 'grip0')->('le

++ adding return motion to acquired answer ++
try: 95 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
constrained motion tried: True
branching: 8->113 (2.14/30.0 s, steps/err: 3(87.1810913086 ms)/0.000706890631916)
end
result: 90 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
end
error: 1.1
constrained motion tried: True
try: 113 - ('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
branching: 90->114 (2.16/30.0 s, steps/err: 53(318.351984024 ms)/1.74994733459e-05)
result: 43 - ('door_frame', 'h1', 'r2', 'grip0')->('door_frame', 'h1', 'r1', 'grip0') = success
end
Goal reached
branching: 43->115 (2.17/30.0 s, steps/err: 53(274.334907532 ms)/2.59094317883e-05)
constrained motion tried: True
end
constrained motion tried: True
end
result: 94 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0') = success
try: 114 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
constrain

try: 98 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
try: 136 - ('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
error: 0.73
error: 0.73
end
Goal reached
constrained motion tried: True
result: 105 - ('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r2', 'grip0') = success
branching: 105->139 (2.42/30.0 s, steps/err: 53(297.407865524 ms)/2.15773764868e-05)
======================= terminated 12: required answers acquired  (2.4/30.0) ===============================
joint min
Goal reached
constrained motion tried: False
======================= terminated 9: Stop called from other agent  (2.4/30.0) ===============================
Motion Plan Failure
result: 77 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
======================= terminated 16: Stop called from other agent  (2.5/30.0) ===============================
Goal reached
joint min
constrained motion tried: False
Motion Plan Failure
result: 77 - (

joint min
constrained motion tried: False
Motion Plan Failure
result: 102 - ('lever', 'h0', 'r2', 'grip0')->('lever', 'h2', 'r2', 'grip0') = fail
======================= terminated 1: Stop called from other agent  (2.8/30.0) ===============================
========================== FINISHED (2.8 / 30.0 s) ==============================]


In [18]:
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)

In [19]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r1', 'knob')
('door_frame', 'h1', 'r1', 'knob')->('door_frame', 'h1', 'r1', 'grip0')
('door_frame', 'h1', 'r1', 'grip0')->('door_frame', 'h1', 'r2', 'grip0')
('door_frame', 'h1', 'r2', 'grip0')->('lever', 'h1', 'r2', 'grip0')
('lever', 'h1', 'r2', 'grip0')->('lever', 'h0', 'r2', 'grip0')
('lever', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r2', 'grip0')
('door_frame', 'h0', 'r2', 'grip0')->('door_frame', 'h0', 'r1', 'grip0')
('door_frame', 'h0', 'r1', 'grip0')->('door_frame', 'h0', 'r1', 'knob')
('door_frame', 'h0', 'r1', 'knob')->('door_frame', 'h0', 'r1', 'knob')
